## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
unique_counts =  application_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff = 500
application_types_to_replace = list(application_type_counts[application_type_counts < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
classification_counts_greater_than_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE

cutoff = 1000
classifications_to_replace = list(classification_counts[classification_counts < cutoff].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_encoded = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_encoded.drop('IS_SUCCESSFUL', axis=1).values
y = application_encoded['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize the Model

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model - deep neural net
model = Sequential()

# Add the first hidden layer with 100 neurons and 'relu' activation
model.add(Dense(units=100, activation='relu', input_dim=X_train.shape[1]))

# Add a second hidden layer with 50 neurons and 'relu' activation
model.add(Dense(units=50, activation='relu'))

# Add a third hidden layer with 25 neurons and 'relu' activation
model.add(Dense(units=25, activation='relu'))

# Add the output layer with 'sigmoid' activation for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with 'binary_crossentropy' loss function, 'adam' optimizer, and 'accuracy' metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with increased number of epochs (200)
model.fit(X_train_scaled, y_train, epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.5696 - accuracy: 0.7195
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7294
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7311
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7318
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7330
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7336
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7327
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7333
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5464 - accuracy: 0.7339
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5458 - accura

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5809 - accuracy: 0.7285 - 365ms/epoch - 1ms/step
Loss: 0.5809337496757507, Accuracy: 0.7285131216049194


In [16]:
!pip install keras-tuner
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define the model-building function for the Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_dim=X_train.shape[1]))
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])

    return model

# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory='my_dir',
    project_name='alphabet_soup')

# Perform the hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Trial 3 Complete [00h 12m 46s]
val_accuracy: 0.7318950593471527

Best val_accuracy So Far: 0.7318950593471527
Total elapsed time: 00h 27m 44s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 416)               18304     
                                                                 
 dense_1 (Dense)             (None, 416)               173472    
                                                                 
 dense_2 (Dense)             (None, 1)                 417       
                                                                 
Total params: 192,193
Trainable params: 192,193
Non-trainable params: 0
_________________________________________________________________


In [17]:

# Evaluate the best model
best_model_loss, best_model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {best_model_loss}, Accuracy: {best_model_accuracy}")

268/268 - 1s - loss: 0.5580 - accuracy: 0.7320 - 842ms/epoch - 3ms/step
Loss: 0.5579742193222046, Accuracy: 0.7320116758346558


In [19]:
best_model.save('AlphabetSoupCharity_Optimization.h5')